In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import os
import numpy as np
from functools import partial
from fastai import datasets
from pathlib import Path

from exp.nlp import\
 

from exp.utils import\
    ListContainer,\
    ItemList,\
    _get_files,\
    get_files, SplitData,\
    random_splitter,\
    Processor,\
    compose,\
    label_by_func
    
from exp.data import\
    DataLoader,\
    DataBunch,\
    CategoryProcessor,\
    grandparent_splitter,\
    parent_labeler

## Data 

In [3]:
path = datasets.untar_data(datasets.URLs.IMDB)

In [4]:
path.ls()

[PosixPath('/Users/ricardofernandez/.fastai/data/imdb/ld.pkl'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/tmp_clas'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/unsup'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/README'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/tmp_lm'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/ll_clas.pkl'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/train')]

In [5]:
def read_file(fn):
    with open(fn, 'r', encoding='utf8') as f: return f.read()
    
class TextList(ItemList):
    @classmethod
    def from_files(cls, path, extensions='.txt', recurse=True, include=None, **kwargs):
        return cls(get_files(path, extensions, recurse=recurse, include=include), path, **kwargs)
    
    def get(self, i):
        if isinstance(i, Path): return read_file(i)
        return i
    

In [6]:
a = []
for p in os.scandir(path):
    if p.is_file():
        a.append(p.name)
        print(p.name)

ld.pkl
imdb.vocab
README
ll_clas.pkl


In [7]:
path

PosixPath('/Users/ricardofernandez/.fastai/data/imdb')

In [8]:
x = list(os.walk(path/'test'))
x[0]

('/Users/ricardofernandez/.fastai/data/imdb/test',
 ['neg', 'pos'],
 ['labeledBow.feat'])

In [9]:
(path/"test").ls()

[PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/pos'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/labeledBow.feat')]

In [10]:
_get_files(path,a,'.txt')

[]

In [11]:
get_files(path, [], True, include=["Train"])

[PosixPath('/Users/ricardofernandez/.fastai/data/imdb/ld.pkl'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/README'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/ll_clas.pkl')]

In [12]:
get_files(path, [".txt"], recurse=True, include="train")[:3]

[PosixPath('/Users/ricardofernandez/.fastai/data/imdb/train/neg/1821_4.txt'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/train/neg/10402_1.txt'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/train/neg/1062_4.txt')]

In [13]:
p.is_file()

False

In [14]:
path.ls()

[PosixPath('/Users/ricardofernandez/.fastai/data/imdb/ld.pkl'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/tmp_clas'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/unsup'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/README'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/tmp_lm'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/ll_clas.pkl'),
 PosixPath('/Users/ricardofernandez/.fastai/data/imdb/train')]

In [15]:
for i, (p,d,f) in enumerate(os.walk(path)):
    if i == 0:
        d[:] = [o for o in d if o in ["train"]]
    print(p)

/Users/ricardofernandez/.fastai/data/imdb
/Users/ricardofernandez/.fastai/data/imdb/train
/Users/ricardofernandez/.fastai/data/imdb/train/neg
/Users/ricardofernandez/.fastai/data/imdb/train/pos


In [16]:
for i, (p,d,f) in enumerate(os.walk(path)):
    if i == 0:
        d[:] = [o for o in d if o in ["train"]]
    print(p)

/Users/ricardofernandez/.fastai/data/imdb
/Users/ricardofernandez/.fastai/data/imdb/train
/Users/ricardofernandez/.fastai/data/imdb/train/neg
/Users/ricardofernandez/.fastai/data/imdb/train/pos


In [17]:
il = TextList.from_files(path, include=['train', 'test', 'unsup'])

In [18]:
il

TextList (100000 items)
[PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/1821_4.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/9487_1.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/4604_4.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/2828_2.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/10890_1.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/3351_4.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/8070_2.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/1027_4.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/8248_3.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/4290_4.txt')...]
Path: /Users/ricardofernandez/.fastai/data/imdb

In [19]:
len(il), len(il.items)

(100000, 100000)

In [20]:
txt = il[0]
txt

"Alan Rickman & Emma Thompson give good performances with southern/New Orleans accents in this detective flick. It's worth seeing for their scenes- and Rickman's scene with Hal Holbrook. These three actors mannage to entertain us no matter what the movie, it seems. The plot for the movie shows potential, but one gets the impression in watching the film that it was not pulled off as well as it could have been. The fact that it is cluttered by a rather uninteresting subplot and mostly uninteresting kidnappers really muddles things. The movie is worth a view- if for nothing more than entertaining performances by Rickman, Thompson, and Holbrook."

In [21]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [22]:
sd

SplitData
Train: TextList (90005 items)
[PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/1821_4.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/9487_1.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/4604_4.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/2828_2.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/10890_1.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/8070_2.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/1027_4.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/8248_3.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/4290_4.txt'), PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/10096_1.txt')...]
Path: /Users/ricardofernandez/.fastai/data/imdb

Valid: TextList (9995 items)
[PosixPath('/Users/ricardofernandez/.fastai/data/imdb/test/neg/3351_4.txt'), PosixPath('/Users/ricardofernandez/.fas

## Tokenizing

In [23]:
import spacy, html, re
from typing import Collection

In [24]:
#special tokens
UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxrep xxwrep xxup xxmaj".split()

def sub_br(t):
    "Replaces the <br /> by \n"
    re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
    return re_br.sub("\n", t)

def spec_add_spaces(t):
    "Add spaces around / and #"
    return re.sub(r'([/#])', r' \1 ', t)

def rm_useless_spaces(t):
    "Remove multiple spaces"
    return re.sub(' {2,}', ' ', t)

def replace_rep(t):
    "Replace repetitions at the character level: cccc -> TK_REP 4 c"
    def _replace_rep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    re_rep = re.compile(r'(\S)(\1{3,})')
    return re_rep.sub(_replace_rep, t)
    
def replace_wrep(t):
    "Replace word repetitions: word word word -> TK_WREP 3 word"
    def _replace_wrep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_WREP} {len(cc.split())+1} {c} '
    re_wrep = re.compile(r'(\b\w+\W+)(\1{3,})')
    return re_wrep.sub(_replace_wrep, t)

def fixup_text(x):
    "Various messy things we've seen in documents"
    re1 = re.compile(r'  +')
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))
    
default_pre_rules = [fixup_text, replace_rep, replace_wrep, spec_add_spaces, rm_useless_spaces, sub_br]
default_spec_tok = [UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ]

In [25]:
sub_br("Hello < br/>")

'Hello \n'

In [26]:
spec_add_spaces("Hello / Padre #")

'Hello  /  Padre  # '

In [27]:
rm_useless_spaces("Useless     spaces")

'Useless spaces'

In [28]:
replace_rep("leed leeeed")

'leed l xxrep 4 e d'

In [29]:
replace_wrep("be your boy boy boy boy boyfriend")

'be your  xxwrep 4 boy  boyfriend'

In [30]:
x = re.compile(r'[ab]')
x.sub('d','a b c')

'd d c'

In [31]:
replace_rep('cccc')

' xxrep 4 c '

In [32]:
replace_wrep('word word word word word')

' xxwrep 4 word  word'

In [33]:
def replace_all_caps(x):
    "Replace tokens in ALL CAPS by their lower version and add `TK_UP` before."
    res = []
    for t in x:
        if t.isupper() and len(t) > 1: res.append(TK_UP); res.append(t.lower())
        else: res.append(t)
    return res

def deal_caps(x):
    "Replace all Capitalized tokens in by their lower version and add `TK_MAJ` before."
    res = []
    for t in x:
        if t == '': continue
        if t[0].isupper() and len(t) > 1 and t[1:].islower(): res.append(TK_MAJ)
        res.append(t.lower())
    return res

def add_eos_bos(x): return [BOS] + x + [EOS]

default_post_rules = [deal_caps, replace_all_caps, add_eos_bos]

In [34]:
'CAT'.isupper(), "Cat".isupper()

(True, False)

In [35]:
replace_all_caps(["HELLO","Twitter", "WORLD"])

['xxup', 'hello', 'Twitter', 'xxup', 'world']

In [36]:
"I Love Dominica".split()

['I', 'Love', 'Dominica']

In [37]:
deal_caps("I Love Dominica".split())

['i', 'xxmaj', 'love', 'xxmaj', 'dominica']

In [38]:
replace_all_caps("I AM SHOUTING".split())

['I', 'xxup', 'am', 'xxup', 'shouting']

In [39]:
deal_caps("My name is Ricardo".split())

['xxmaj', 'my', 'name', 'is', 'xxmaj', 'ricardo']

In [40]:
#export
from spacy.symbols import ORTH
from concurrent.futures import ProcessPoolExecutor
from fastprogress import progress_bar

def parallel(func, arr, max_workers=4):
    if max_workers<2: results = list(progress_bar(map(func, enumerate(arr)), total=len(arr)))
    else:
        with ProcessPoolExecutor(max_workers=max_workers) as ex:
            return list(progress_bar(ex.map(func, enumerate(arr)), total=len(arr)))
    if any([o is not None for o in results]): return results

In [41]:
def f(x):
    x0, x1 = x
    if x0 == 0:
        return None
    return x1**2

parallel(f, [0, 2, 3], 1)

[None, 4, 9]

In [42]:
any([o is not None for o in [None, None, None]])

False

In [43]:
[o is not None for o in [None, None, None]]

[False, False, False]

In [44]:
ORTH

65

In [45]:
#export
class TokenizeProcessor(Processor):
    def __init__(self, lang="en", chunksize=2000, pre_rules=None, post_rules=None, max_workers=4): 
        self.chunksize,self.max_workers = chunksize,max_workers
        self.tokenizer = spacy.blank(lang).tokenizer
        for w in default_spec_tok:
            self.tokenizer.add_special_case(w, [{ORTH: w}])
        self.pre_rules  = default_pre_rules  if pre_rules  is None else pre_rules
        self.post_rules = default_post_rules if post_rules is None else post_rules
        
    def proc_chunk(self, args):
        i,chunk = args
        chunk = [compose(t, self.pre_rules) for t in chunk]
        docs = [[d.text for d in doc] for doc in self.tokenizer.pipe(chunk)]
        docs = [compose(t, self.post_rules) for t in docs]
        return docs
    
    def __call__(self, items): 
        toks = []
        if isinstance(items[0], Path): items = [read_file(i) for i in items]
        chunks = [items[i: i+self.chunksize] for i in (range(0, len(items), self.chunksize))]
        toks = parallel(self.proc_chunk, chunks, max_workers=self.max_workers)
        return sum(toks, [])
    
    def proc1(self, item): return self.proc_chunk([item])[0]
    
    def deprocess(self, toks): return [self.deproc1(tok) for tok in toks]
    def deproc1(self, tok):    return " ".join(tok)

In [46]:
sum([['a', 'b'], ['c'], ['d', 'e']], [])

['a', 'b', 'c', 'd', 'e']

In [47]:
tokenizer = spacy.blank('en').tokenizer
for doc in tokenizer.pipe(['car', 'fat', 'does not', 'Mr.', "doesn't"]):
    for d in doc:
        print(d.text)

car
fat
does
not
Mr.
does
n't


In [48]:
tp = TokenizeProcessor()

In [49]:
txt[:250]

"Alan Rickman & Emma Thompson give good performances with southern/New Orleans accents in this detective flick. It's worth seeing for their scenes- and Rickman's scene with Hal Holbrook. These three actors mannage to entertain us no matter what the mo"

In [50]:
" ".join(["cat", "bat"])

'cat bat'

In [51]:
tp(il[:10])[0][:10]

['xxbos',
 'xxmaj',
 'alan',
 'xxmaj',
 'rickman',
 '&',
 'xxmaj',
 'emma',
 'xxmaj',
 'thompson']

In [52]:
' • '.join(tp(il[:100])[0])[:400]

"xxbos • xxmaj • alan • xxmaj • rickman • & • xxmaj • emma • xxmaj • thompson • give • good • performances • with • southern • / • xxmaj • new • xxmaj • orleans • accents • in • this • detective • flick • . • xxmaj • it • 's • worth • seeing • for • their • scenes- • and • xxmaj • rickman • 's • scene • with • xxmaj • hal • xxmaj • holbrook • . • xxmaj • these • three • actors • mannage • to • ente"

## Numericalizing

In [53]:
import collections
from collections import Counter

class NumericalizeProcessor(Processor):
    def __init__(self, vocab=None, max_vocab=60000, min_freq=2): 
        self.vocab,self.max_vocab,self.min_freq = vocab,max_vocab,min_freq
    
    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            freq = Counter(p for o in items for p in o)
            self.vocab = [o for o,c in freq.most_common(self.max_vocab) if c >= self.min_freq]
            for o in reversed(default_spec_tok):
                if o in self.vocab: self.vocab.remove(o)
                self.vocab.insert(0, o)
        if getattr(self, 'otoi', None) is None:
            self.otoi = collections.defaultdict(int,{v:k for k,v in enumerate(self.vocab)}) 
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return [self.otoi[o] for o in item]
    
    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    def deproc1(self, idx): return [self.vocab[i] for i in idx]

In [54]:
default_spec_tok

['xxunk', 'xxpad', 'xxbos', 'xxeos', 'xxrep', 'xxwrep', 'xxup', 'xxmaj']

In [55]:
proc_tok,proc_num = TokenizeProcessor(max_workers=3),NumericalizeProcessor()

In [56]:
%time ll = label_by_func(sd, lambda x: 0, proc_x = [proc_tok,proc_num])

CPU times: user 21.3 s, sys: 11.3 s, total: 32.6 s
Wall time: 2min 9s


In [57]:
type(ll)

exp.utils.SplitData

In [58]:
type(ll.train)

exp.utils.LabeledData

In [59]:
ll.train[0]

([2,
  7,
  1869,
  7,
  9732,
  205,
  7,
  3733,
  7,
  4442,
  221,
  67,
  374,
  27,
  2455,
  124,
  7,
  184,
  7,
  5770,
  2607,
  17,
  19,
  1209,
  504,
  9,
  7,
  16,
  22,
  294,
  335,
  28,
  79,
  39794,
  11,
  7,
  9732,
  22,
  150,
  27,
  7,
  4257,
  7,
  16479,
  9,
  7,
  151,
  297,
  170,
  0,
  14,
  2804,
  200,
  74,
  520,
  64,
  8,
  29,
  10,
  16,
  209,
  9,
  7,
  8,
  130,
  28,
  8,
  29,
  293,
  1042,
  10,
  30,
  43,
  240,
  8,
  1454,
  17,
  168,
  8,
  31,
  20,
  16,
  25,
  37,
  1981,
  141,
  26,
  88,
  26,
  16,
  95,
  41,
  99,
  9,
  7,
  8,
  212,
  20,
  16,
  15,
  16290,
  47,
  12,
  269,
  2736,
  3297,
  11,
  679,
  2736,
  11867,
  83,
  39795,
  198,
  9,
  7,
  8,
  29,
  15,
  294,
  12,
  0,
  63,
  28,
  179,
  68,
  93,
  437,
  374,
  47,
  7,
  9732,
  10,
  7,
  4442,
  10,
  11,
  7,
  16479,
  9,
  3],
 0)

In [60]:
ll.train.x_obj(0)

"xxbos xxmaj alan xxmaj rickman & xxmaj emma xxmaj thompson give good performances with southern / xxmaj new xxmaj orleans accents in this detective flick . xxmaj it 's worth seeing for their scenes- and xxmaj rickman 's scene with xxmaj hal xxmaj holbrook . xxmaj these three actors xxunk to entertain us no matter what the movie , it seems . xxmaj the plot for the movie shows potential , but one gets the impression in watching the film that it was not pulled off as well as it could have been . xxmaj the fact that it is cluttered by a rather uninteresting subplot and mostly uninteresting kidnappers really muddles things . xxmaj the movie is worth a xxunk if for nothing more than entertaining performances by xxmaj rickman , xxmaj thompson , and xxmaj holbrook . xxeos"

In [61]:
ll.train.y_obj(0)

0

In [62]:
ll.train.proc_y is None

True

In [63]:
" ".join(ll.train.proc_x[1].deproc1(ll.train[0][0]))

"xxbos xxmaj alan xxmaj rickman & xxmaj emma xxmaj thompson give good performances with southern / xxmaj new xxmaj orleans accents in this detective flick . xxmaj it 's worth seeing for their scenes- and xxmaj rickman 's scene with xxmaj hal xxmaj holbrook . xxmaj these three actors xxunk to entertain us no matter what the movie , it seems . xxmaj the plot for the movie shows potential , but one gets the impression in watching the film that it was not pulled off as well as it could have been . xxmaj the fact that it is cluttered by a rather uninteresting subplot and mostly uninteresting kidnappers really muddles things . xxmaj the movie is worth a xxunk if for nothing more than entertaining performances by xxmaj rickman , xxmaj thompson , and xxmaj holbrook . xxeos"

In [64]:
import pickle

In [65]:
pickle.dump(ll, open(path/'ld.pkl', 'wb'))

In [66]:
ll = pickle.load(open(path/'ld.pkl', 'rb'))

## Batching 

In [67]:
from IPython.display import display, HTML
import pandas as pd

In [68]:
stream = """
In this notebook, we will go back over the example of classifying movie reviews we studied in part 1 and dig deeper under the surface. 
First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the Processor used in the data block API.
Then we will study how we build a language model and train it.\n
"""
tokens = np.array(tp([stream])[0])

In [69]:
tokens[:50]

array(['xxbos', '\n', 'xxmaj', 'in', 'this', 'notebook', ',', 'we', 'will', 'go', 'back', 'over', 'the', 'example',
       'of', 'classifying', 'movie', 'reviews', 'we', 'studied', 'in', 'part', '1', 'and', 'dig', 'deeper', 'under',
       'the', 'surface', '.', '\n', 'xxmaj', 'first', 'we', 'will', 'look', 'at', 'the', 'processing', 'steps',
       'necessary', 'to', 'convert', 'text', 'into', 'numbers', 'and', 'how', 'to', 'customize'], dtype='<U11')

In [70]:
tokens.shape

(91,)

In [71]:
tokens[0], tokens[-3], tokens[-2], tokens[-1]

('xxbos', '.', '\n\n', 'xxeos')

In [72]:
tokens.shape, tokens.shape[0]/15

((91,), 6.066666666666666)

In [73]:
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,\n,xxmaj,in,this,notebook,",",we,will,go,back,over,the,example,of
classifying,movie,reviews,we,studied,in,part,1,and,dig,deeper,under,the,surface,.
\n,xxmaj,first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into
numbers,and,how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have
another,example,of,the,xxmaj,processor,used,in,the,data,block,api,.,\n,xxmaj
then,we,will,study,how,we,build,a,language,model,and,train,it,.,\n\n


notice that last batch was not taken

In [74]:
bs,bptt = 6,5
for k in range(3):
    d_tokens = np.array([tokens[i*seq_len + k*bptt:i*seq_len + (k+1)*bptt] for i in range(bs)])
    df = pd.DataFrame(d_tokens)
    display(HTML(df.to_html(index=False,header=None)))

xxbos,\n,xxmaj,in,this
classifying,movie,reviews,we,studied
\n,xxmaj,first,we,will
numbers,and,how,to,customize
another,example,of,the,xxmaj
then,we,will,study,how


notebook,",",we,will,go
in,part,1,and,dig
look,at,the,processing,steps
it,.,xxmaj,by,doing
processor,used,in,the,data
we,build,a,language,model


back,over,the,example,of
deeper,under,the,surface,.
necessary,to,convert,text,into
this,",",we,'ll,have
block,api,.,\n,xxmaj
and,train,it,.,\n\n


In [75]:
import torch
from torch import tensor

In [76]:
class LM_PreLoader():
    def __init__(self, data, bs=64, bptt=70, shuffle=False):
        self.data,self.bs,self.bptt,self.shuffle = data,bs,bptt,shuffle
        total_len = sum([len(t) for t in data.x])
        self.n_batch = total_len // bs
        self.batchify()
    
    def __len__(self): return ((self.n_batch-1) // self.bptt) * self.bs
    
    def __getitem__(self, idx):
        source = self.batched_data[idx % self.bs]
        seq_idx = (idx // self.bs) * self.bptt
        return source[seq_idx:seq_idx+self.bptt],source[seq_idx+1:seq_idx+self.bptt+1]
    
    def batchify(self):
        # place all docs as single stream
        texts = self.data.x
        if self.shuffle: texts = texts[torch.randperm(len(texts))]
        stream = torch.cat([tensor(t) for t in texts])
        self.batched_data = stream[:self.n_batch * self.bs].view(self.bs, self.n_batch)

In [77]:
dl = DataLoader(LM_PreLoader(ll.valid, shuffle=True), batch_size=64)

In [78]:
len(ll.valid) # of documents in valid

9995

In [79]:
# number of tokens per batch, all docs concat
d = dl.dataset
d.n_batch

46991

In [80]:
# concat all docs and brocken them into n_batch chuncks by batch size
# not exact as we are doing integer division
sum([len(t) for t in ll.valid.x]), d.n_batch*d.bs

(3007443, 3007424)

In [81]:
# data in batch form
d.batched_data.shape, (d.bs, d.n_batch)

(torch.Size([64, 46991]), (64, 46991))

In [82]:
# the 4613 is broken into segments of bptt, total number of instances
((d.n_batch-1)//d.bptt)*d.bs, len(d), len(d)/d.bs, d.bs

(42944, 42944, 671.0, 64)

In [83]:
# with our new number of instances 42112 we break into batches of size
# 64 creating 658 batches of of size (64, 70)
len(dl), len(dl.dataset), dl.dataset.n_batch

(671, 42944, 46991)

In [84]:
len(dl)*dl.batch_size

42944

In [85]:
iter_dl = iter(dl)
x1,y1 = next(iter_dl)
x2,y2 = next(iter_dl)

In [86]:
x1.size(), y1.size(), x2.size(), y2.size()

(torch.Size([64, 70]),
 torch.Size([64, 70]),
 torch.Size([64, 70]),
 torch.Size([64, 70]))

In [87]:
x1[0,:10], y1[0,0:10]

(tensor([   2,   18,   87,   37,   41,    8, 3458,   13,  129,  496]),
 tensor([  18,   87,   37,   41,    8, 3458,   13,  129,  496,   14]))

Let's check it all works ok: x1, y1, x2 and y2 should all be of size bs by bptt. The texts in each row of x1 should continue in x2. y1 and y2 should have the same texts as their x counterpart, shifted of one position to the right.

In [88]:
vocab = proc_num.vocab

In [89]:
" ".join(vocab[o] for o in x1[0])

"xxbos i did not have the fortune of being able to view xxmaj burn xxmaj after xxmaj reading in the theatre , but having watched it on dvd , i can say that i 'm glad that i waited . xxmaj its not that the movie was bad , but there were several scenes that i had to go back and re - watch a number of times because they"

In [90]:
" ".join(vocab[o] for o in y1[0])

"i did not have the fortune of being able to view xxmaj burn xxmaj after xxmaj reading in the theatre , but having watched it on dvd , i can say that i 'm glad that i waited . xxmaj its not that the movie was bad , but there were several scenes that i had to go back and re - watch a number of times because they were"

In [91]:
" ".join(vocab[o] for o in x2[0])

"were just so damned funny . xxmaj the trailers and tv spots for the film would have you believe that this yet another goofy xxmaj brad xxmaj pitt film where he runs around like a pretty boy for ninety minutes , but do n't be fooled , this film is far more gruesome and much funnier that i believed . xxmaj like any typical film by the xxmaj brothers xxmaj"

In [92]:
#export
def get_lm_dls(train_ds, valid_ds, bs, bptt, **kwargs):
    return (DataLoader(LM_PreLoader(train_ds, bs, bptt, shuffle=True), batch_size=bs, **kwargs),
            DataLoader(LM_PreLoader(valid_ds, bs, bptt, shuffle=False), batch_size=2*bs, **kwargs))

def lm_databunchify(sd, bs, bptt, **kwargs):
    return DataBunch(*get_lm_dls(sd.train, sd.valid, bs, bptt, **kwargs))

In [93]:
bs, bptt = 64, 70
data = lm_databunchify(ll, bs, bptt)

## Batching for classification

In [205]:
proc_cat = CategoryProcessor()

In [219]:
il = TextList.from_files(path, include=['train', 'test'])
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='test'))
ll = label_by_func(sd, parent_labeler, proc_x = [proc_tok, proc_num], proc_y=proc_cat)

In [220]:
pickle.dump(ll, open(path/'ll_clas.pkl', 'wb'))

In [221]:
ll = pickle.load(open(path/'ll_clas.pkl', 'rb'))

In [222]:
[(ll.train.x_obj(i), ll.train.y_obj(i)) for i in [1,12552]]

[('xxbos xxmaj well ... tremors i , the original started off in 1990 and i found the movie quite enjoyable to watch . however , they proceeded to make tremors ii and iii . xxmaj trust me , those movies started going downhill right after they finished the first one , i mean , ass blasters ? ? ? xxmaj now , only xxmaj god himself is capable of answering the question " why in xxmaj gods name would they create another one of these dumpster dives of a movie ? " xxmaj tremors iv can not be considered a bad movie , in fact it can not be even considered an epitome of a bad movie , for it lives up to more than that . xxmaj as i attempted to sit though it , i noticed that my eyes started to bleed , and i hoped profusely that the little girl from the ring would crawl through the tv and kill me . did they really think that dressing the people who had stared in the other movies up as though they we \'re from the wild west would make the movie ( with the exact same occurrences ) any better ? honestl

In [223]:
from torch.utils.data import Sampler

class SortSampler(Sampler):
    def __init__(self, data_source, key): self.data_source,self.key = data_source,key
    def __len__(self): return len(self.data_source)
    def __iter__(self):
        return iter(sorted(list(range(len(self.data_source))), key=self.key, reverse=True))

In [224]:
#export
class SortishSampler(Sampler):
    def __init__(self, data_source, key, bs):
        self.data_source,self.key,self.bs = data_source,key,bs

    def __len__(self) -> int: return len(self.data_source)

    def __iter__(self):
        idxs = torch.randperm(len(self.data_source))
        megabatches = [idxs[i:i+self.bs*50] for i in range(0, len(idxs), self.bs*50)]
        sorted_idx = torch.cat([tensor(sorted(s, key=self.key, reverse=True)) for s in megabatches])
        batches = [sorted_idx[i:i+self.bs] for i in range(0, len(sorted_idx), self.bs)]
        max_idx = torch.argmax(tensor([self.key(ck[0]) for ck in batches]))  # find the chunk with the largest key,
        batches[0],batches[max_idx] = batches[max_idx],batches[0]            # then make sure it goes first.
        batch_idxs = torch.randperm(len(batches)-2)
        sorted_idx = torch.cat([batches[i+1] for i in batch_idxs]) if len(batches) > 1 else LongTensor([])
        sorted_idx = torch.cat([batches[0], sorted_idx, batches[-1]])
        return iter(sorted_idx)

In [251]:
from torch import LongTensor
def pad_collate(samples, pad_idx=1, pad_first=False):
    max_len = max([len(s[0]) for s in samples])
    res = torch.zeros(len(samples), max_len).long() + pad_idx
    for i,s in enumerate(samples):
        if pad_first: res[i, -len(s[0]):] = LongTensor(s[0])
        else:         res[i, :len(s[0]) ] = LongTensor(s[0])
    return res, tensor([s[1] for s in samples])

In [252]:
bs = 64
train_sampler = SortishSampler(ll.train.x, key=lambda t: len(ll.train[int(t)][0]), bs=bs)
train_dl = DataLoader(ll.train, batch_size=bs, sampler=train_sampler, collate_fn=pad_collate)

In [253]:
x = next(iter(train_sampler))

In [254]:
x = torch.tensor(10)

In [255]:
type(x)

torch.Tensor

In [256]:
isinstance(x, torch.Tensor) and x.shape==[]

False

In [257]:
type(x.shape)

torch.Size

In [258]:
x.shape == torch.Size([])

True

In [259]:
x.shape

torch.Size([])

In [260]:
x.size() is []

False

In [261]:
len(ll.train)

25000

In [262]:
iter_dl = iter(train_dl)
x,y = next(iter_dl)

In [263]:
len(x), len(y)

(64, 64)

In [264]:
for i in range(5):
    print(ll.train[i][1])

0
0
0
0
0


In [265]:
lengths = []
for i in range(x.size(0)): lengths.append(x.size(1) - (x[i]==1).sum().item())
lengths[:5], lengths[-1]

([3311, 1513, 1417, 1417, 1371], 1019)

In [271]:
x.size(1)

257

In [277]:
x,y = next(iter_dl)
lengths = []
for i in range(x.size(0)):
    lengths.append(x.size(1) - (x[i]==1).sum().item())
lengths[:10], lengths[-1]

([110, 110, 110, 109, 109, 109, 109, 109, 108, 108], 95)

In [268]:
x

tensor([[    2,    18,   234,  ...,  7067,     9,     3],
        [    2,     7,  1483,  ..., 48832,     9,     3],
        [    2,    18,   369,  ...,    16,    50,     3],
        ...,
        [    2,    18,    58,  ...,     1,     1,     1],
        [    2,    88,    10,  ...,     1,     1,     1],
        [    2,     7,    19,  ...,     1,     1,     1]])

In [278]:
#export
def get_clas_dls(train_ds, valid_ds, bs, **kwargs):
    train_sampler = SortishSampler(train_ds.x, key=lambda t: len(train_ds.x[t]), bs=bs)
    valid_sampler = SortSampler(valid_ds.x, key=lambda t: len(valid_ds.x[t]))
    return (DataLoader(train_ds, batch_size=bs, sampler=train_sampler, collate_fn=pad_collate, **kwargs),
            DataLoader(valid_ds, batch_size=bs*2, sampler=valid_sampler, collate_fn=pad_collate, **kwargs))

def clas_databunchify(sd, bs, **kwargs):
    return DataBunch(*get_clas_dls(sd.train, sd.valid, bs, **kwargs))

In [279]:
bs,bptt = 64,70
data = clas_databunchify(ll, bs)